# Integrating Flotorch Session Management with Autogen

This notebook demonstrates how to integrate Flotorch's session management with an Autogen agent. By using `FlotorchAutogenSession` as the storage backend for the agent's conversational context, we can create a stateful agent that remembers the history of the current session.

### Prerequesit
Configure model, API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Concepts:
- **`FlotorchAutogenLLM`**: The language model powering the agent.
- **`FlotorchAutogenSession`**: The session storage mechanism for persisting conversation history.
- **`model_context`**: The agent parameter used to pass the session history, enabling short-term memory.
- **`AssistantAgent`, `RoundRobinGroupChat`**: The core components for building and orchestrating the AI workflow.

## 1. Setup and Configuration

The following cells install the necessary packages, configure API credentials, and import all required components from the Flotorch and Autogen libraries.

In [ ]:
# install flotorch autogen package
%pip install flotorch[autogen]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"

In [ ]:
# Import Autpgen, Flotorch and related modules
import datetime
from flotorch.autogen.llm import FlotorchAutogenLLM
from flotorch.autogen.sessions import FlotorchAutogenSession
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# for supressing the unwanted logs from autogen
import logging
logging.disable(logging.WARNING)

print("Imported necessary libraries successfully")

## 2. Model and Session Configuration

Here, we initialize `FlotorchAutogenLLM` as the reasoning engine for our agent. We then create an instance of `FlotorchAutogenSession` which will manage the conversational history for the agent.

In [ ]:
model  =  FlotorchAutogenLLM(
        model_id=FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL,
    )

print(f"Flotorch LLM model configured")

In [ ]:
session_memory = FlotorchAutogenSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

print("session_memory is initialized")

## 3. Agent and Team Configuration

We define the `AssistantAgent` and a `RoundRobinGroupChat` to manage the interaction. The `session_memory` instance is passed to the agent's `model_context` parameter, which enables the agent to access the history of the current chat session.

In [ ]:
assistant = AssistantAgent(
        name="FlotorchAssistant",
        model_client=model,
        model_context = session_memory,
        system_message = """  
                        You are a helpful Assistant with access to the session memory, 
                        so use the session memory properly and answer the user queries.
                        """ ,
        )
    
team = RoundRobinGroupChat(
        participants=[assistant],
        max_turns=1
    )
print(f"Agent and Task created successfully: {assistant.name}")

## 4. Interactive Chat

This loop starts an interactive chat session. The agent will use the Flotorch-backed session memory to recall context from previous messages within this conversation. Type 'exit' to end the session.

In [ ]:
while True:
    user_input = input("user: ")

    if user_input.lower().strip() == "exit":
        break

    stream = team.run_stream(task = user_input)
    result = await Console(stream)


## Summary

This notebook successfully demonstrated how to create a conversational agent with short-term memory. 
By integrating **Flotorch's session management** with **Autogen**, we achieved a **stateful interaction** where the agent can recall information from earlier in the conversation.

### Key Achievements

- **Session Persistence** 
  Configured `FlotorchAutogenSession` as the backend for the agent's `model_context`, enabling it to maintain context across turns.

- **Context-Aware Responses** 
  The agent successfully referenced previously mentioned details (e.g., the user's name, their dog's name) to provide more natural and relevant replies.

- **Streamlined Integration** 
  Demonstrated a clear and effective method to connect Flotorch's robust session infrastructure with Autogen’s powerful agent framework.